In [1]:
import pandas as pd 
import numpy as np
import networkx as nx 
import json 
import csv 
import matplotlib as plt 
import cdlib
import os


In [2]:
G = list()


for y in range(2012,2021):
   
    g = nx.read_weighted_edgelist(f'../data/years/{str(y)}.weighted.edgelist')
    G.append(g)


maxW = 0
for g in G:
    weights = [g.edges[edge]['weight'] for edge in g.edges()]
    if max(weights) > maxW : maxW = max(weights) 

for g in G:
    for edge in g.edges():
        g.edges[edge]['dissimilarity'] =maxW + 1 - g.edges[edge]['weight']

In [3]:


for g in G:
    weights = [g.edges[edge]['weight'] for edge in g.edges()]
    maxW = max(weights) 
    for edge in g.edges():
        g.edges[edge]['dissimilarity'] =maxW + 1 - g.edges[edge]['weight']

    

In [4]:
for e in list(g.edges)[:10]: print(e)

('0', '1979')
('0', '2238')
('0', '2648')
('0', '8615')
('0', '8841')
('0', '10252')
('0', '11278')
('0', '12564')
('0', '13935')
('0', '14005')


In [5]:
def coreNode(g,alfa,weight):
    
    
    def localCloseNeighborThreshold(g,p,weight):     
        geomin = 1   
        for q in g.neighbors(p): geomin *= g.edges[(p,q)][weight]
        return geomin ** (1 / len(list(g.neighbors(p))))
    
    def localCloseNeighbors(g,p,tresholds,weight):
        return [ q for q in g.neighbors(p) if g.edges[(p,q)][weight] <= tresholds[p] or g.edges[(p,q)][weight] <= tresholds[q] ]
    
        
    
    localCloseNeighborThresholds = {p:localCloseNeighborThreshold(g,p,weight) for p in g.nodes()}
    
    corenodes = list()
    for node in g.nodes:
        
        localMinimumClusteringThreshold = alfa * len(list(g.neighbors(node)))
        treshold = localCloseNeighborThresholds[node]
        
        if len(localCloseNeighbors(g,node,localCloseNeighborThresholds,weight)) >= localMinimumClusteringThreshold: 
            
            corenodes.append(node)
        
    return corenodes

In [10]:
allcores = list()
for g in G:
    allcores.append(coreNode(g,0.9,'dissimilarity'))

In [11]:
for i, cores in enumerate(allcores): 
    print(len(G[i].edges()),' ',len(G[i].nodes()),' ',len(cores))

19305   3013   1496
56023   4949   2132
85569   6399   2702
90204   8148   2698
92469   9451   2643
93468   10256   3531
104198   10846   3278
98616   11397   4583
75629   8818   4342


In [27]:
def dyCores(G,alfa=0.9,weight='weight'):
    
    communities = list()
    
    t = 0 
    for g in G:
        print(t)
        t+=1
        core = coreNode(g,alfa,weight)
        for node in g.nodes():
            g.nodes[node]['mark'] = 'unvisited'
            if node in core: g.nodes[node]['core']=True 
            else : g.nodes[node]['core']=False
     
        community = dict()
        k = 0
        
        
        for p in g.nodes():
            
            if g.nodes[p]['mark'] == 'visited' : continue
            if g.nodes[p]['core'] == False :
                g.nodes[p]['mark'] = 'border'
                continue
                
            candidates = set(g.neighbors(p))
            community[k] = list(p)
            
            
            while len(candidates) != 0:
                
                q = candidates.pop()
                
                if g.nodes[q]['core'] == True :
                    
                    if g.nodes[q]['mark'] != 'visited':
                        g.nodes[q]['mark'] = 'visited'
                        community[k].append(q)
                    
                else:
                        
                    for x in g.neighbors(q):
                        if g.nodes[x]['mark'] == 'unvisited' : candidates.add(x)
                        elif  g.nodes[x]['mark'] == 'border' : community[k].append(x) 
                        
                        community[k].append(q)
                        g.nodes[q]['mark'] = 'visited'
                        
            k = k + 1
               
        #OUTLIERS are the last community, are the remaining border nodes
        community[k] = list(node for node in g.nodes() if g.nodes[node]['mark'] == 'border')  
       
                    
        communities.append(community)
    
    return communities
                        
                    
                    
                
            
            
        
        
        
        
        

In [32]:
def dyCores(G,alfa=0.9,weight='weight'):
    
    communities = list()
    
    t = 0 
    for g in G:
        print(t)
        t+=1
        core = coreNode(g,alfa,weight)
        for node in g.nodes():
            g.nodes[node]['mark'] = 'unvisited'
            if node in core: g.nodes[node]['core']=True 
            else : g.nodes[node]['core']=False
     
        community = dict()
        k = 0
        
        
        for p in g.nodes():
            
            if g.nodes[p]['mark'] == 'visited' : continue
            if g.nodes[p]['core'] == False :
                g.nodes[p]['mark'] = 'border'
                continue
                
            candidates = set(g.neighbors(p))
            community[k] = list(p)
            
            
            while len(candidates) != 0:
                
                q = candidates.pop()
                
                if g.nodes[q]['core'] == True :
                    
                    if g.nodes[q]['mark'] != 'visited':
                        g.nodes[q]['mark'] = 'visited'
                        community[k].append(q)
                    
                    else:

                        for x in g.neighbors(q):
                            if g.nodes[x]['mark'] == 'unvisited' : candidates.add(x)
                            elif  g.nodes[x]['mark'] == 'border' : community[k].append(x) 

                            community[k].append(q)
                            g.nodes[q]['mark'] = 'visited'

            k = k + 1
               
        #OUTLIERS are the last community, are the remaining border nodes
        community[k] = list(node for node in g.nodes() if g.nodes[node]['mark'] == 'border')  
       
                    
        communities.append(community)
    
    return communities
                        
                    
                    
                
            
            
        
        
        
        
        

In [33]:
comms = dyCores(G,0.9,'dissimilarity')

0
1
2
3
4
5
6
7
8


In [34]:
for comm in comms:
    print([len(c) for i,c in comm.items()])

[6, 6, 16, 13, 5, 7, 5, 9, 5, 5, 5, 5, 143, 5, 9, 9, 496, 4, 5, 4, 5, 5, 5, 5, 7, 61, 6, 83, 8, 8, 5, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 4, 4, 4, 7, 6, 6, 5, 5, 5, 5, 6, 5, 6, 33, 5, 5, 5, 21, 4, 4, 4, 5, 5, 7, 6, 51, 29, 4, 4, 4, 5, 5, 5, 5, 6, 6, 5, 5, 5, 6, 6, 5, 7, 5, 31, 30, 63, 5, 5, 5, 36, 43, 5, 5, 5, 33, 5, 5, 4, 48, 13, 44, 6, 5, 20, 5, 5, 4, 27, 26, 7, 84, 107, 5, 4, 18, 31, 4, 30, 3, 85, 4, 5, 4, 5, 44, 5, 4, 4, 5, 5, 27, 26, 4, 44, 40, 4, 32, 4, 4, 4, 4, 4, 23, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 43, 43, 4, 4, 5, 5, 5, 5, 35, 28, 8, 23, 5, 5, 7, 23, 5, 5, 32, 30, 5, 26, 6, 55, 5, 4, 4, 5, 4, 5, 4, 31, 4, 33, 5, 24, 5, 52, 4, 4, 5, 42, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 27, 5, 48, 5, 7, 5, 51, 5, 3, 5, 5, 5, 7, 4, 4, 4, 5, 5, 5, 44, 4, 5, 5, 5, 5, 5, 5, 38, 4, 4, 4, 3, 39, 4, 5, 5, 5, 24, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 35, 5, 5, 19, 4, 5, 4, 4, 4, 4, 4, 5, 5, 39, 4, 79, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 5, 5, 6, 5, 5, 28